# 🚀 Showcase: 2D to 360° Video Generation

This notebook provides a comprehensive showcase for the 'Beyond the Frame' project. It will guide you through setting up the environment and then offer two distinct ways to demo the technology:

*   **Option 1: Interactive Demo** - Launch a web interface to upload your own videos.
*   **Option 2: Automated Processing** - Automatically download a video from a URL and process it.

**Instructions:**
1. Make sure your runtime is set to use a GPU (`Runtime` > `Change runtime type` > `T4 GPU`).
2. Run the 'Common Setup' cells first.
3. Choose **one** of the two demo options and run its corresponding cells.

### Step 1: Common Setup (Run These Cells First)

In [ ]:
print('Cloning repository...')
!git clone --recurse-submodules https://github.com/Red-Fairy/argus-code
%cd argus-code

print('Installing dependencies...')
# Install faiss-gpu first to prevent errors, then install other requirements.
!pip install -q faiss-gpu
!pip install -q -r requirements.txt
!pip install -q yt-dlp gdown

print('Cloning and installing asmk...')
!git clone https://github.com/jenicek/asmk
%cd asmk/cython/
!cythonize *.pyx
%cd ..
!pip install .
%cd ..
print('All dependencies installed.')

In [ ]:
print('Downloading pre-trained models...')
!mkdir -p checkpoints
!gdown --id 1mZpViQY2yvwav-CcxdsQouYu8t52b25G -O checkpoints/pretrained-weights.zip
!unzip -q checkpoints/pretrained-weights.zip -d checkpoints/

!mkdir -p mega-sam/Depth-Anything/checkpoints
!wget -q https://huggingface.co/spaces/LiheYoung/Depth-Anything/resolve/main/checkpoints/depth_anything_vitl14.pth -O mega-sam/Depth-Anything/checkpoints/depth_anything_vitl14.pth

!mkdir -p mega-sam/cvd_opt
!gdown --id 1sWDsfuZ3Up38EUQt7-JDTT1HcGHuJgvT -O mega-sam/cvd_opt/raft-things.pth
print('Models downloaded successfully.')

### Step 2: Choose Your Demo

**Important:** Only run the cells for the option you want to use.

---

#### Option 1: Interactive Demo (Gradio Web UI)

Run the cell below to launch a Gradio web interface. A public URL will be generated. Click it to open the demo in a new tab, where you can upload your own video for processing.

In [ ]:
!python gradio_demo.py --share

---

#### Option 2: Automated Processing from a URL

Run the two cells below to automatically download a video from a URL and process it using the command-line script.

In [ ]:
# --- You can change this URL to any video you want to process ---
video_url = 'https://www.youtube.com/watch?v=YE7VzlLtp-4' # Big Buck Bunny (placeholder)

!mkdir -p input_videos
!yt-dlp -o 'input_videos/video.mp4' -f 'best[height<=720]' {video_url}

video_path = 'input_videos/video.mp4'
print(f'Video downloaded to: {video_path}')

In [ ]:
print('Starting automated processing...')
unet_path = './checkpoints/pretrained-weights'
save_folder = './outputs'

!python inference.py \
    --unet_path {unet_path} \
    --video_path {video_path} \
    --val_save_folder {save_folder} \
    --guidance_scale 3 \
    --num_inference_steps 25

print(f'\n--- Processing Complete! ---\n')
print(f'Check the `{save_folder}` directory for your generated 360° video.')